This notebook creates a NN to make predictions based on pre-existing data.

In [83]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
import pandas as pd
import oracledb
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [84]:
import yaml
from pathlib import Path
home = str(Path.home())

def process_yaml():
	with open("../config.yaml") as file:
		return yaml.safe_load(file)

class OracleJSONDatabaseThickConnection:
    def __init__(self, data=process_yaml()):
        oracledb.init_oracle_client(lib_dir=data['INSTANT_CLIENT'])
        self.pool = oracledb.create_pool(user=data['db']['username'], password=data['db']['password'], dsn=data['db']['dsn'],
            min=1, max=4, increment=1, getmode=oracledb.POOL_GETMODE_WAIT)
        print('Connection successful.')



    def close_pool(self):
        self.pool.close()
        print('Connection pool closed.')



    def insert(self, collection_name, json_object_to_insert):
        connection = self.pool.acquire()
        connection.autocommit = True
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)

        try:
            x_collection.insertOne(json_object_to_insert)
            print('[DBG] INSERT {} OK'.format(json_object_to_insert))
        except cx_Oracle.IntegrityError as e:
            print('[DBG] INSERT {} ERR: {} '.format(json_object_to_insert, e))
            return -1
        self.pool.release(connection)
        return 1


    def delete(self, collection_name, on_column, on_value):
        connection = self.pool.acquire()
        connection.autocommit = True
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)
        qbe = {on_column: on_value}
        x_collection.find().filter(qbe).remove()
        self.pool.release(connection)


    def get_connection(self):
        connection = self.pool.acquire()
        connection.autocommit = True
        return connection


    def close_connection(self, conn_object):
        self.pool.release(conn_object)


    def get_collection_names(self):
        connection = self.pool.acquire()
        connection.autocommit = True
        returning_object = connection.getSodaDatabase().getCollectionNames(startName=None, limit=0)
        self.pool.release(connection)
        return returning_object

    def open_collection(self, collection_name):
        connection = self.pool.acquire()
        returning_object = self.pool.acquire().getSodaDatabase().openCollection(collection_name)
        self.pool.release(connection)
        return returning_object

In [85]:
db = OracleJSONDatabaseThickConnection()
print(db.get_collection_names())

Connection successful.
['1v1_model', 'match', 'match_detail', 'matchups', 'predictor', 'predictor_liveclient', 'summoner']


In [86]:
data = db.open_collection('predictor')
all_data = list()
i = 0
for doc in data.find().getCursor():
    content = doc.getContent()
    all_data.append(content)
    i+= 1
    if i > 100000:
        break

print('Data length: {}'.format(len(all_data)))

Data length: 100001


In [87]:
df = pd.read_json(json.dumps(all_data), orient='records')

df.head(5)

,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
0,0,0,1334,42,6705,0,0,0,240060,0,...,5088,123,165,58,767,37,1453,6574,4,0
1,0,70,1097,30,5054,0,0,0,240060,0,...,4697,117,119,53,659,38,822,4750,3,0
2,0,0,1507,30,13840,0,0,0,240060,0,...,865,112,88,410,851,45,963,1711,3,0
3,1,0,1169,41,10571,5,0,0,240060,0,...,4559,105,308,0,766,37,1075,4559,3,0
4,1,0,1313,42,43737,0,24,0,240060,0,...,9185,119,550,0,762,44,1190,11795,4,0


In [88]:
df.columns

Index(['winner', 'trueDamageTaken', 'totalGold', 'magicResist',
       'timeEnemySpentControlled', 'ccReduction', 'goldPerSecond', 'spellVamp',
       'timestamp', 'bonusArmorPenPercent', 'magicPenPercent', 'participantId',
       'armorPenPercent', 'physicalDamageTaken', 'magicPen', 'powerMax',
       'identifier', 'totalDamageTaken', 'power', 'attackDamage', 'lifesteal',
       'abilityPower', 'powerRegen', 'trueDamageDoneToChampions',
       'jungleMinionsKilled', 'physicalDamageDoneToChampions', 'movementSpeed',
       'bonusMagicPenPercent', 'cooldownReduction',
       'totalDamageDoneToChampions', 'minionsKilled', 'healthRegen', 'health',
       'trueDamageDone', 'magicDamageDone', 'physicalDamageDone',
       'attackSpeed', 'magicDamageTaken', 'magicDamageDoneToChampions',
       'healthMax', 'armor', 'xp', 'totalDamageDone', 'level', 'armorPen'],
      dtype='object')

In [89]:
df.describe()

,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
count,100001.000000,100001.000000,100001.000000,100001.000000,1.000010e+05,100001.000000,100001.000000,100001.000000,1.000010e+05,100001.0,...,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.0
mean,0.499435,414.964400,5367.997520,45.037440,1.034584e+05,6.518115,16.332257,0.076339,8.676975e+05,0.0,...,28631.072959,141.885171,2935.984230,2724.485435,1373.901351,68.981020,5912.046050,49200.524065,8.397946,0.0
std,0.500002,723.207542,3926.266767,21.065325,1.405437e+05,12.510065,34.592667,1.184294,5.609666e+05,0.0,...,40271.852413,45.227910,3481.799732,4754.082157,614.553940,37.755724,4624.107962,52616.841484,4.424767,0.0
min,0.000000,0.000000,500.000000,7.000000,0.000000e+00,-30.000000,0.000000,0.000000,0.000000e+00,0.0,...,0.000000,56.000000,0.000000,0.000000,340.000000,17.000000,0.000000,0.000000,1.000000,0.0
25%,0.000000,0.000000,2061.000000,33.000000,1.083400e+04,0.000000,0.000000,0.000000,4.200990e+05,0.0,...,3077.000000,112.000000,401.000000,98.000000,888.000000,45.000000,2019.000000,8725.000000,5.000000,0.0
50%,0.000000,150.000000,4704.000000,38.000000,5.458100e+04,0.000000,0.000000,0.000000,8.402340e+05,0.0,...,10046.000000,125.000000,1703.000000,829.000000,1292.000000,59.000000,5165.000000,29384.000000,9.000000,0.0
75%,1.000000,509.000000,7927.000000,47.000000,1.402290e+05,10.000000,10.000000,0.000000,1.260467e+06,0.0,...,38959.000000,156.000000,4273.000000,3098.000000,1729.000000,81.000000,9020.000000,76705.000000,12.000000,0.0
max,1.000000,18181.000000,23396.000000,492.000000,2.520831e+06,77.000000,233.000000,37.000000,2.760816e+06,0.0,...,368403.000000,621.000000,35552.000000,66384.000000,5793.000000,693.000000,26694.000000,391159.000000,18.000000,0.0


In [90]:
# drop columns we don't want
df = df.drop(columns=['identifier', 'participantId', 'timestamp'])

#train = df.sample(frac=0.8, random_state=200) #random state is a seed value
#test = df.drop(train.index)
label = 'winner'

X = df.drop(columns=[label])
y = df.pop(label)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=1, test_size=0.2)

print(X_train.columns)

Index(['trueDamageTaken', 'totalGold', 'magicResist',
       'timeEnemySpentControlled', 'ccReduction', 'goldPerSecond', 'spellVamp',
       'bonusArmorPenPercent', 'magicPenPercent', 'armorPenPercent',
       'physicalDamageTaken', 'magicPen', 'powerMax', 'totalDamageTaken',
       'power', 'attackDamage', 'lifesteal', 'abilityPower', 'powerRegen',
       'trueDamageDoneToChampions', 'jungleMinionsKilled',
       'physicalDamageDoneToChampions', 'movementSpeed',
       'bonusMagicPenPercent', 'cooldownReduction',
       'totalDamageDoneToChampions', 'minionsKilled', 'healthRegen', 'health',
       'trueDamageDone', 'magicDamageDone', 'physicalDamageDone',
       'attackSpeed', 'magicDamageTaken', 'magicDamageDoneToChampions',
       'healthMax', 'armor', 'xp', 'totalDamageDone', 'level', 'armorPen'],
      dtype='object')


In [96]:
model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, kernel_initializer='normal')
])

loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=100, batch_size=128)

Epoch 1/100
625/625 [==============================] - 1s 783us/step - loss: 120403.0938 - accuracy: 0.5088
Epoch 2/100
625/625 [==============================] - 0s 773us/step - loss: 671.1675 - accuracy: 0.5069
Epoch 3/100
625/625 [==============================] - 0s 769us/step - loss: 410.5541 - accuracy: 0.5052
Epoch 4/100
625/625 [==============================] - 0s 775us/step - loss: 350.9062 - accuracy: 0.5065
Epoch 5/100
625/625 [==============================] - 0s 777us/step - loss: 438.2362 - accuracy: 0.5088
Epoch 6/100
625/625 [==============================] - 0s 777us/step - loss: 1013.6039 - accuracy: 0.5078
Epoch 7/100
625/625 [==============================] - 0s 785us/step - loss: 372.9247 - accuracy: 0.5045
Epoch 8/100
625/625 [==============================] - 0s 773us/step - loss: 1324.9822 - accuracy: 0.5062
Epoch 9/100
625/625 [==============================] - 0s 775us/step - loss: 273.3281 - accuracy: 0.5059
Epoch 10/100
625/625 [============================

In [ ]:
save_path = '/home/ubuntu/models/autogluon_trained_models'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train, time_limit=600)

In [ ]:
y_test = test[label]  # values to predict
test_data_nolabel = test.drop(columns=[label])  # delete label column to prove we're not cheating, also drop identifier column
test_data_nolabel.head(5)

In [ ]:
predictor = TabularPredictor.load(save_path)

y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


In [ ]:
predictor.leaderboard(test, silent=False)

In [ ]:
predictor.feature_importance(test)

In [ ]:
print('Finished')